In [2]:
#install required libs which is not in google colab originlly
!pip install colorama
!pip install lightgbm
!git clone https://github.com/analokmaus/kuma_utils.git
!pip install category_encoders
!pip install feature-engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'kuma_utils'...
remote: Enumerating objects: 940, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 940 (delta 120), reused 118 (delta 105), pack-reused 795
Receiving objects: 100% (940/940), 684.29 KiB | 1.05 MiB/s, done.
Resolving deltas: 100% (608/608), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 KB 7.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer,OneHotEncoder, StandardScaler
from category_encoders import MEstimateEncoder
import joblib
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression, HuberRegressor
import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer
from sklearn.metrics import roc_auc_score, accuracy_score
from feature_engine.selection import SelectByShuffling
import pickle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv('drive/MyDrive/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('drive/MyDrive/tabular-playground-series-aug-2022/test.csv')
test_id = pd.read_csv('drive/MyDrive/tabular-playground-series-aug-2022/test.csv') 

data = pd.concat([train,test],axis=0).reset_index(drop=True) #preprocess trainin and test data at the same time
data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8) # add some features according to data pattern survey
data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
data['loading'] = np.log1p(data['loading']) # compress the values of loading feature
train_y = train['failure'].astype(int) #store the ground truth of training data
data=data.drop('id', axis=1)

flaot_features = [feature for feature in data.columns if data[feature].dtype==float]
flaot_features.remove('failure')

code_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
measurement_17_high_cor = {}

for Pcode in code_list:
  measurement_17_high_cor[Pcode] = []
  corr = data[flaot_features][data.product_code==Pcode].corr()
  for i in range(len(corr["measurement_17"])-1) : #measurement_17 and failure must be ignored
    if round(corr["measurement_17"][i],1) >= 0.2: # if the correlation between the measurement and measurement_17 is big enough, then consider it when inferencing values of measurement_17
      measurement_17_high_cor[Pcode].append(flaot_features[i])

for Pcode in code_list: # Use high correlation measurements to inference measurement_17 with Regression method
  single_code_data = data[data.product_code==Pcode]
  inference_17_col = measurement_17_high_cor[Pcode].copy()
  inference_17_col.append('measurement_17')
  inference_data_17 = single_code_data[inference_17_col].dropna()
  inference_target_17 = single_code_data[(single_code_data[measurement_17_high_cor[Pcode]].isnull().sum(axis=1)==0)]
  inference_target_17 = inference_target_17[inference_target_17['measurement_17'].isnull()]
  model_17 = HuberRegressor()
  model_17.fit(inference_data_17[measurement_17_high_cor[Pcode]], inference_data_17['measurement_17'])
  data.loc[(data.product_code==Pcode)&(data[measurement_17_high_cor[Pcode]].isnull().sum(axis=1)==0)&(data['measurement_17'].isnull()),'measurement_17'] = model_17.predict(inference_target_17[measurement_17_high_cor[Pcode]])

for Pcode in code_list: #impute lost measurement values according to other measurements of the same product_code with an imputer
  model_inpute = LGBMImputer()
  data.loc[data.product_code==Pcode, flaot_features] = model_inpute.fit_transform(data.loc[data.product_code==Pcode, flaot_features])

train = data[data.failure.notnull()]
test = data[data.failure.isnull()]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [6]:
train_x = train.drop(['failure'], axis=1)
data_col = train_x.columns
train_y = train['failure'].astype(int)
test_x = test.drop(['failure'], axis=1)


encoder = MEstimateEncoder() #used to encode features with strings
train_x = encoder.fit_transform(train_x, train_y)
test_x = encoder.transform(test_x)


power = PowerTransformer() #used to transform data to make data more Gaussian-like
train_x = power.fit_transform(train_x)
test_x = power.transform(test_x)

scaler = StandardScaler() #standardize features and scaling to unit variance
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

train_x = pd.DataFrame(train_x, columns=data_col)
test_x = pd.DataFrame(test_x, columns=data_col)

log_1 = LogisticRegression(solver='saga', random_state=42) #use LogisticRegression to inference predictions

feature_sel = SelectByShuffling(variables=None, estimator=log_1, scoring='roc_auc', threshold=0, cv=5, random_state=49) #find out which features are the most important
feature_sel.fit(train_x, train_y)
feature_importance_order = pd.Series(feature_sel.performance_drifts_).sort_values(ascending=False).index.tolist()
feature_importance_order

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


['loading',
 'attribute_2',
 'attribute_3',
 'attribute_1',
 'product_code',
 'attribute_0',
 'measurement_17',
 'measurement_4',
 'm5_missing',
 'measurement_7',
 'measurement_8',
 'measurement_6',
 'measurement_10',
 'measurement_15',
 'measurement_9',
 'measurement_16',
 'measurement_2',
 'm3_missing',
 'measurement_11',
 'measurement_13',
 'measurement_3',
 'measurement_1',
 'measurement_12',
 'measurement_5',
 'measurement_0',
 'measurement_14']

In [17]:
#After preprocessing the test data (blocks above), now we can use the pretrained model to inference predictions

pkl_file_path = "drive/MyDrive/tabular-playground-series-aug-2022/0816170_model.pkl"  
log_1 = pd.read_pickle(pkl_file_path) # load the trained model

submission_path = "drive/MyDrive/tabular-playground-series-aug-2022/0816170.csv"  
pred=log_1.predict_proba(test_x[feature_importance_order[0:10]])[:, 1]
submission = pd.DataFrame({"id":test_id['id'], "failure":pred})

with open(submission_path, 'w', encoding = 'utf-8-sig') as f: # output submission
  submission.to_csv(f, index=False)